## I. Share data with ThreadPoolExecutor

ThreadPoolExecutor come from the concurrent.futures module. It has a simpler interface that the threading module and basically do what the multithreading but in a better way. We can use:
- executor.submit in a for loop. It creates an iterable where results are stored as they come
- execut.map. It creates an iterable where results come in the starting order

Also something very interesting with these convenient function is that error will only be raised when we go through the iterator which mean that we can manage the error flow properly.

In [3]:
import sys

module_path = '..'
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import perf_decorator

In [4]:
import concurrent.futures
from time import sleep
from numpy import random

In [5]:
def do_something(seconds, raised_sec=None):
    sleep(seconds)
    if raised_sec and seconds == raised_sec:
        raise ValueError(f"Cannot accept sleeping for {raised_sec} s.")
    return f'Done {seconds} seconds ...'

### a. using a loop: result came in the finished order

In [7]:
@perf_decorator
def main():
    
    # the pool made the decision on how to affect worker
    with concurrent.futures.ThreadPoolExecutor() as executor:
        secs = [5, 3, 2, 1]

        results = [executor.submit(do_something, sec, raised_sec=2) for sec in secs]
    
        # iterator than we can loop over that will yield result as completed
        for f in concurrent.futures.as_completed(results):
            try:
                print(f.result())
            except ValueError as err:
                print(f"Error has been properly managed: {err}")

    return results
    
main()

Done 1 seconds ...
Error has been properly managed: Cannot accept sleeping for 2 s.
Done 3 seconds ...
Done 5 seconds ...
main execution time 5.00s


[<Future at 0x2b87b00fb90 state=finished returned str>,
 <Future at 0x2b87a195d00 state=finished returned str>,
 <Future at 0x2b879a2bb00 state=finished raised ValueError>,
 <Future at 0x2b87b025670 state=finished returned str>]

Threads come in order from the fastest to slowest.

### b. using a loop: result came in the finished order

In [10]:
@perf_decorator
def main():
    
    # the pool made the decision on how to affect worker
    with concurrent.futures.ThreadPoolExecutor() as executor:
        secs = [5, 3, 2, 1]

        results = executor.map(do_something, secs)
    
        # iterator than we can loop over that will yield result in execution order
        for res in results:
            print(res)

    return results
    
main()

Done 5 seconds ...
Done 3 seconds ...
Done 2 seconds ...
Done 1 seconds ...
main execution time 5.01s


<generator object Executor.map.<locals>.result_iterator at 0x000002B87AFE7C40>

Threads come in execution order.